In [ ]:
from pytket.circuit import Circuit, CircBox, QControlBox

from pytket.circuit.display import render_circuit_jupyter

from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus

from pytket.backends.backendresult import BackendResult
from pytket.extensions.qiskit import AerStateBackend

import phayes

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Constructs unitary for an arbitrary size of qubits
def construct_unitary(n_qubits: int, theta) -> Circuit():
    ### Wait on Carson and Christine ###
    u_circ = Circuit(n_qubits)
    u_circ.U1(theta, 0)
    return u_circ

In [15]:
def build_phase_estimation_circuit(
    m: int, beta: float, state_prep_circuit: Circuit, unitary_circuit: Circuit
) -> Circuit:
    # Define a Circuit with a measurement and prep register
    qpe_circ: Circuit = Circuit()
    n_state_prep_qubits = state_prep_circuit.n_qubits
    measurement_register = qpe_circ.add_q_register("m", 1)
    state_prep_register = qpe_circ.add_q_register("p", n_state_prep_qubits)
    qpe_circ.add_circuit(state_prep_circuit, list(state_prep_register))

    # Add Hadamard gate to the measurement register
    qpe_circ.H(measurement_register[0])
    # Create a controlled unitary with a single control qubit
    unitary_circuit.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)
    # Run the controlled unitary m times
    for _ in range(m):
        qpe_circ.add_qcontrolbox(
            controlled_u_gate, list(measurement_register) + list(state_prep_register))

    qpe_circ.U1(beta/np.pi, measurement_register[0])
    qpe_circ.H(measurement_register[0])
    qpe_circ.measure_register(measurement_register, "c")
    return qpe_circ

In [16]:
def get_next_digit(m: int, beta: float, state_prep_circuit: Circuit, unitary_circuit: Circuit,
                   backend=AerStateBackend(), n_shots=1000):
    qpe_circ = build_phase_estimation_circuit(m, beta, state_prep_circuit, unitary_circuit)
    compiled_circ = backend.get_compiled_circuit(qpe_circ)
    result = backend.run_circuit(compiled_circ, n_shots)
    sorted_shots = result.get_counts().most_common()
    print(sorted_shots)
    return sorted_shots[0][0][0]

In [17]:
# Prepares circuit by notting all the qubits
def prepare_circ(circ: Circuit) -> Circuit():
    n_qubits = circ.n_qubits
    prepped_circ = Circuit(n_qubits)
    
    for i in range(n_qubits):
        prepped_circ.X(i)
    return prepped_circ

In [18]:
def iterated_qpe(state_prep_circuit: Circuit, unitary_circuit: Circuit, precision: int):
    digits = []
    for i in range(precision-1,-1,-1):
        omega = 0
        for j in range(len(digits)):
            omega = omega - np.pi*digits[j]/2**(j+1)
        x = get_next_digit(2**i, omega, state_prep_circuit, unitary_circuit)
        digits.insert(0,x)
    bitstring = "".join([str(bit) for bit in digits])
    print(bitstring)
    integer_j = int(bitstring, 2)

    # Calculate theta estimate
    return integer_j / (2 ** len(bitstring))

In [19]:
# MAIN 

test_qubits = 4
theta = np.pi / 4

test_circ = Circuit(test_qubits)
prepped_circ = prepare_circ(test_circ)

test_u = construct_unitary(test_qubits, theta)
estimated_phase = iterated_qpe(prepped_circ, test_u, 5)

print("Estimated Phase:", estimated_phase)
print("True Phase:", theta / 2)

TypeError: U2(): incompatible function arguments. The following argument types are supported:
    1. (self: pytket._tket.circuit.Circuit, angle0: typing.Union[sympy.Expr, float], angle1: typing.Union[sympy.Expr, float], qubit: int, **kwargs) -> pytket._tket.circuit.Circuit
    2. (self: pytket._tket.circuit.Circuit, angle0: typing.Union[sympy.Expr, float], angle1: typing.Union[sympy.Expr, float], qubit: pytket._tket.unit_id.Qubit, **kwargs) -> pytket._tket.circuit.Circuit

Invoked with: [H m[0]; X p[0]; X p[1]; X p[2]; X p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3]; qif (m[0] = 1) CircBox p[0], p[1], p[2], p[3